In [3]:
import matplotlib.pyplot as plt
from pandas import DataFrame, Series
import pandas as pd
import numpy as np

# GroupBy Mechanics

In [2]:
df = DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5),
                'data2' : np.random.randn(5)})
df

,data1,data2,key1,key2
0,-0.877282,2.238228,a,one
1,3.119975,-0.507464,a,two
2,-0.070169,0.652689,b,one
3,0.025651,0.269067,b,two
4,0.041929,0.438204,a,one


In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
grouped.mean()

key1
a    0.761540
b   -0.022259
Name: data1, dtype: float64

In [6]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.417677
      two     3.119975
b     one    -0.070169
      two     0.025651
Name: data1, dtype: float64

In [7]:
means.unstack()

key2,one,two
key1,,
a,-0.417677,3.119975
b,-0.070169,0.025651


In [8]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    3.119975
            2006   -0.070169
Ohio        2005   -0.425815
            2006    0.041929
Name: data1, dtype: float64

In [9]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.761540,0.722989
b,-0.022259,0.460878


In [10]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one  -0.417677  1.338216
     two   3.119975 -0.507464
b    one  -0.070169  0.652689
     two   0.025651  0.269067

In [11]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## <span style="color:blue">Iterating Over Groups</span>

In [12]:
for name, group in df.groupby('key1'):
    print name
    print group

a
      data1     data2 key1 key2
0 -0.877282  2.238228    a  one
1  3.119975 -0.507464    a  two
4  0.041929  0.438204    a  one
b
      data1     data2 key1 key2
2 -0.070169  0.652689    b  one
3  0.025651  0.269067    b  two


In [13]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print k1, k2
    print group

a one
      data1     data2 key1 key2
0 -0.877282  2.238228    a  one
4  0.041929  0.438204    a  one
a two
      data1     data2 key1 key2
1  3.119975 -0.507464    a  two
b one
      data1     data2 key1 key2
2 -0.070169  0.652689    b  one
b two
      data1     data2 key1 key2
3  0.025651  0.269067    b  two


In [14]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,data1,data2,key1,key2
2,-0.070169,0.652689,b,one
3,0.025651,0.269067,b,two


## <span style="color:blue">Selecting a Column or Subset of Columns</span>

In [15]:
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]

In [16]:
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

In [17]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   1.338216
     two  -0.507464
b    one   0.652689
     two   0.269067

In [19]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [20]:
s_grouped.mean()

key1  key2
a     one     1.338216
      two    -0.507464
b     one     0.652689
      two     0.269067
Name: data2, dtype: float64

## <span style="color:blue">Grouping with dicts and series</span>

In [24]:
people = DataFrame(np.random.randn(5, 5),
                   columns=['a', 'b', 'c', 'd', 'e'],
                   index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.ix[2:3, ['b', 'c']] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,-1.032808,-0.480299,0.617450,-0.901594,-1.617187
Steve,-0.061053,1.002932,1.089380,-1.592453,-0.497825
Wes,-0.028115,NaN,NaN,0.853028,0.929697
Jim,-0.926632,1.680364,0.679247,-2.362683,-0.971424
Travis,1.073693,0.748046,-1.947836,-0.893465,-0.550290


In [25]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [26]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-0.284144,-3.130295
Steve,-0.503073,0.444053
Wes,0.853028,0.901582
Jim,-1.683436,-0.217693
Travis,-2.841300,1.271450


In [31]:
people.ix['Joe'][['a','b','e']].sum()

-3.130294941780218

In [33]:
map_series = Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [34]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


## <span style="color:blue">Grouping with functions</span>

In [36]:
people.groupby(len).sum()

,a,b,c,d,e
3,-1.987555,1.200065,1.296697,-2.411250,-1.658915
5,-0.061053,1.002932,1.089380,-1.592453,-0.497825
6,1.073693,0.748046,-1.947836,-0.893465,-0.550290


In [37]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.032808 -0.480299  0.617450 -0.901594 -1.617187
  two -0.926632  1.680364  0.679247 -2.362683 -0.971424
5 one -0.061053  1.002932  1.089380 -1.592453 -0.497825
6 two  1.073693  0.748046 -1.947836 -0.893465 -0.550290

## <span style="color:blue">Grouping by Index Levels</span>

In [40]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]], names=['cty', 'tenor'])
hier_df = DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.204007  0.298005  1.016455 -0.387248  0.759645
1      1.801451  0.156768 -0.941581 -0.590860 -1.467784
2      1.091742  1.340418 -0.608127  0.754351 -1.156529
3     -0.127962 -0.042645 -0.091781  1.032460 -0.360382

In [41]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# Data Aggregation

In [42]:
df

,data1,data2,key1,key2
0,-0.877282,2.238228,a,one
1,3.119975,-0.507464,a,two
2,-0.070169,0.652689,b,one
3,0.025651,0.269067,b,two
4,0.041929,0.438204,a,one


In [43]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    2.504366
b    0.016069
Name: data1, dtype: float64

In [44]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,3.997257,2.745692
b,0.095821,0.383622


In [45]:
grouped.describe()

data1     data2
key1                          
a    count  3.000000  3.000000
     mean   0.761540  0.722989
     std    2.093537  1.394824
     min   -0.877282 -0.507464
     25%   -0.417677 -0.034630
     50%    0.041929  0.438204
     75%    1.580952  1.338216
     max    3.119975  2.238228
b    count  2.000000  2.000000
     mean  -0.022259  0.460878
     std    0.067755  0.271262
     min   -0.070169  0.269067
     25%   -0.046214  0.364972
     50%   -0.022259  0.460878
     75%    0.001696  0.556784
     max    0.025651  0.652689

## <span style="color:blue">Column-wise and Multiple Function Application</span>

In [52]:
tips = pd.read_csv('https://github.com/wesm/pydata-book/raw/master/ch08/tips.csv')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [67]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.tail()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204
243,18.78,3.00,Female,No,Thur,Dinner,2,0.159744


 I’ll group the  tipsby  sexand  smoker

In [49]:
grouped = tips.groupby(['sex', 'smoker'])

In [50]:
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

sex     smoker
Female  No        0.156921
        Yes       0.182150
Male    No        0.160669
        Yes       0.152771
Name: tip_pct, dtype: float64

In [51]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
sex    smoker                                  
Female No      0.156921  0.036421      0.195876
       Yes     0.182150  0.071595      0.360233
Male   No      0.160669  0.041849      0.220186
       Yes     0.152771  0.090588      0.674707

if you pass a list of  (name, function) tuples...

In [52]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
sex    smoker                    
Female No      0.156921  0.036421
       Yes     0.182150  0.071595
Male   No      0.160669  0.041849
       Yes     0.152771  0.090588

In [53]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
                count      mean       max      count       mean    max
sex    smoker                                                         
Female No          54  0.156921  0.252672         54  18.105185  35.83
       Yes         33  0.182150  0.416667         33  17.977879  44.30
Male   No          97  0.160669  0.291990         97  19.791237  48.33
       Yes         60  0.152771  0.710345         60  22.284500  50.81

In [54]:
result['tip_pct']

count      mean       max
sex    smoker                           
Female No         54  0.156921  0.252672
       Yes        33  0.182150  0.416667
Male   No         97  0.160669  0.291990
       Yes        60  0.152771  0.710345

Now, suppose you wanted to apply potentially different functions to one or more of
the columns. <br />
The trick is to pass a dict to  agg that contains a mapping of column names
to any of the function specifications listed so far

In [55]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
sex    smoker            
Female No       5.2   140
       Yes      6.5    74
Male   No       9.0   263
       Yes     10.0   150

In [56]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                    min       max      mean       std  sum
sex    smoker                                             
Female No      0.056797  0.252672  0.156921  0.036421  140
       Yes     0.056433  0.416667  0.182150  0.071595   74
Male   No      0.071804  0.291990  0.160669  0.041849  263
       Yes     0.035638  0.710345  0.152771  0.090588  150

## <span style="color:blue">Returning Aggregated Data in “unindexed” Form</span>

In [59]:
tips.groupby(['sex', 'smoker']).mean()

total_bill       tip      size   tip_pct
sex    smoker                                          
Female No       18.105185  2.773519  2.592593  0.156921
       Yes      17.977879  2.931515  2.242424  0.182150
Male   No       19.791237  3.113402  2.711340  0.160669
       Yes      22.284500  3.051167  2.500000  0.152771

In [57]:
tips.groupby(['sex', 'smoker'], as_index=False).mean()

,sex,smoker,total_bill,tip,size,tip_pct
0,Female,No,18.105185,2.773519,2.592593,0.156921
1,Female,Yes,17.977879,2.931515,2.242424,0.182150
2,Male,No,19.791237,3.113402,2.711340,0.160669
3,Male,Yes,22.284500,3.051167,2.500000,0.152771


# Group-wise Operations and Transformations

In [60]:
df

,data1,data2,key1,key2
0,-0.877282,2.238228,a,one
1,3.119975,-0.507464,a,two
2,-0.070169,0.652689,b,one
3,0.025651,0.269067,b,two
4,0.041929,0.438204,a,one


Suppose, instead, we wanted to add a column to a DataFrame containing group means for each index. <br />
One way to do this is to aggregate, then merge

In [61]:
k1_means = df.groupby('key1').mean().add_prefix('mean_')
print k1_means
print '-'*80
print pd.merge(df, k1_means, left_on='key1', right_index=True)

      mean_data1  mean_data2
key1                        
a       0.761540    0.722989
b      -0.022259    0.460878
--------------------------------------------------------------------------------
      data1     data2 key1 key2  mean_data1  mean_data2
0 -0.877282  2.238228    a  one    0.761540    0.722989
1  3.119975 -0.507464    a  two    0.761540    0.722989
4  0.041929  0.438204    a  one    0.761540    0.722989
2 -0.070169  0.652689    b  one   -0.022259    0.460878
3  0.025651  0.269067    b  two   -0.022259    0.460878


transform applies a function to each group, then places the results in the appropriate locations

In [62]:
key = ['one', 'two', 'one', 'two', 'one']
people.groupby(key).mean()

,a,b,c,d,e
one,0.004257,0.133873,-0.665193,-0.314010,-0.412594
two,-0.493843,1.341648,0.884314,-1.977568,-0.734625


In [63]:
people.groupby(key).transform(np.mean)

,a,b,c,d,e
Joe,0.004257,0.133873,-0.665193,-0.314010,-0.412594
Steve,-0.493843,1.341648,0.884314,-1.977568,-0.734625
Wes,0.004257,0.133873,-0.665193,-0.314010,-0.412594
Jim,-0.493843,1.341648,0.884314,-1.977568,-0.734625
Travis,0.004257,0.133873,-0.665193,-0.314010,-0.412594


In [64]:
def demean(arr):
    return arr - arr.mean()

In [66]:
demeaned = people.groupby(key).transform(demean)
demeaned

,a,b,c,d,e
Joe,-1.037065,-0.614173,1.282643,-0.587584,-1.204594
Steve,0.432789,-0.338716,0.205066,0.385115,0.236800
Wes,-0.032372,NaN,NaN,1.167038,1.342290
Jim,-0.432789,0.338716,-0.205066,-0.385115,-0.236800
Travis,1.069437,0.614173,-1.282643,-0.579454,-0.137697


## <span style="color:blue">Apply: General split-apply-combine</span>

In [68]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


In [84]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [85]:
top(tips, n=6)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [86]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

In [87]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
No     Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   
       Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
Yes    Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
       Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   

                  tip_pct  
smoker day                 
No     Fri  94   0.142857  
       Sat  212  0.186220  
       Sun  156  0.103799  
       Thur 142  0.121389  
Yes    Fri  95   0.117750  
       Sat  170  0.196812  
       Sun  182  0.077178  
       Thur 197  0.115982

In [88]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

smoker       
No      count    151.000000
        mean       0.159328
        std        0.039910
        min        0.056797
        25%        0.136906
        50%        0.155625
        75%        0.185014
        max        0.291990
Yes     count     93.000000
        mean       0.163196
        std        0.085119
        min        0.035638
        25%        0.106771
        50%        0.153846
        75%        0.195059
        max        0.710345
Name: tip_pct, dtype: float64

In [90]:
result.unstack('smoker')

smoker,No,Yes
count,151.000000,93.000000
mean,0.159328,0.163196
std,0.039910,0.085119
min,0.056797,0.035638
25%,0.136906,0.106771
50%,0.155625,0.153846
75%,0.185014,0.195059
max,0.291990,0.710345


In [96]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

In [97]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [81]:
tips.tip_pct.apply( "{:.2f}".format ).head()

0    0.06
1    0.16
2    0.17
3    0.14
4    0.15
Name: tip_pct, dtype: object

## <span style="color:blue">Quantile and Bucket Analysis</span>

In [99]:
frame = DataFrame({'data1': np.random.randn(1000),
                   'data2': np.random.randn(1000)})
factor = pd.cut(frame.data1, 4)
factor[:10]

0    (-2.00118, -0.288]
1    (-2.00118, -0.288]
2       (-0.288, 1.425]
3    (-2.00118, -0.288]
4        (1.425, 3.139]
5    (-2.00118, -0.288]
6        (1.425, 3.139]
7        (1.425, 3.139]
8    (-3.721, -2.00118]
9       (-0.288, 1.425]
Name: data1, dtype: category
Categories (4, object): [(-3.721, -2.00118] < (-2.00118, -0.288] < (-0.288, 1.425] < (1.425, 3.139]]

In [101]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
grouped = frame.data2.groupby(factor)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.721, -2.00118]",25.0,2.035123,-0.026782,-1.299026
"(-2.00118, -0.288]",340.0,3.528646,0.000578,-2.658446
"(-0.288, 1.425]",554.0,3.167844,0.001826,-2.744298
"(1.425, 3.139]",81.0,2.244482,0.180533,-1.690860


In [102]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)

In [103]:
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.631970,0.087211,-2.144484
1,100.0,3.392675,-0.151106,-2.658446
2,100.0,3.528646,0.054039,-2.402869
3,100.0,3.167844,0.031543,-2.405844
4,100.0,2.887940,-0.099339,-1.763913
5,100.0,2.940621,0.207083,-2.652639
6,100.0,2.418692,-0.015560,-2.387670
7,100.0,2.462732,-0.074466,-2.744298
8,100.0,2.234666,0.009604,-2.314792


## <span style="color:blue">Example: Filling Missing Values with Group-specific Values</span>

In [104]:
s = Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1    0.864682
2         NaN
3   -0.329941
4         NaN
5   -0.456699
dtype: float64

In [105]:
s.fillna(s.mean())

0    0.026014
1    0.864682
2    0.026014
3   -0.329941
4    0.026014
5   -0.456699
dtype: float64

In [4]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -0.822003
New York     -1.579799
Vermont            NaN
Florida       1.967879
Oregon       -0.169130
Nevada             NaN
California   -0.175942
Idaho              NaN
dtype: float64

In [5]:
data.groupby(group_key).mean()

East   -0.144641
West   -0.172536
dtype: float64

Filling with the mean

In [6]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio         -0.822003
New York     -1.579799
Vermont      -0.144641
Florida       1.967879
Oregon       -0.169130
Nevada       -0.172536
California   -0.175942
Idaho        -0.172536
dtype: float64

Filling with a specific value

In [7]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio         -0.822003
New York     -1.579799
Vermont       0.500000
Florida       1.967879
Oregon       -0.169130
Nevada       -1.000000
California   -0.175942
Idaho        -1.000000
dtype: float64

## <span style="color:blue">Example: Random Sampling and Permutation</span>

In [ ]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (range(1, 11) + [10] * 3) * 4

In [10]:
base_names = ['A'] + range(2, 11) + ['J', 'K', 'Q']
print base_names

['A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'K', 'Q']


In [11]:
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)
deck = Series(card_val, index=cards)
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [12]:
def draw(deck, n=5):
    return deck.take(np.random.permutation(len(deck))[:n])
draw(deck)

AD     1
9C     9
KS    10
AC     1
5S     5
dtype: int64

Suppose you wanted two random cards from each suit.

In [20]:
get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

C  2C     2
   9C     9
D  3D     3
   6D     6
H  9H     9
   4H     4
S  QS    10
   6S     6
dtype: int64

In [31]:
# alternatively
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

JC    10
9C     9
5D     5
JD    10
2H     2
KH    10
AS     1
8S     8
dtype: int64

## <span style="color:blue">Example: Group Weighted Average and Correlation</span>

In [32]:
df = DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
                'data': np.random.randn(8),
                'weights': np.random.rand(8)})
df

,category,data,weights
0,a,-0.872665,0.282000
1,a,0.622619,0.747894
2,a,1.321400,0.516343
3,a,1.658457,0.805247
4,b,-1.662272,0.953729
5,b,-0.007618,0.823127
6,b,-0.458421,0.925033
7,b,-0.065720,0.177718


In [33]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a    0.951452
b   -0.704041
dtype: float64

In [51]:
sum(df[df.category=='a']['data'] * df[df.category=='a']['weights'])/sum(df[df.category=='a']['weights'])

0.951451928892596

# Pivot Tables and Cross-Tabulation

suppose I wanted to compute a table of group means
(the default pivot_tableaggregation type) arranged by sexand smokeron the rows

In [64]:
tips.pivot_table(index=['sex','smoker'])

size       tip  total_bill
sex    smoker                                
Female No      2.592593  2.773519   18.105185
       Yes     2.242424  2.931515   17.977879
Male   No      2.711340  3.113402   19.791237
       Yes     2.500000  3.051167   22.284500

Now, suppose we want to aggregate only tip_pct and size, and additionally group by day.

In [68]:
tips.pivot_table(['tip_pct', 'size'], index=['sex', 'day'], columns='smoker')

tip_pct                size          
smoker             No       Yes        No       Yes
sex    day                                         
Female Fri   0.165296  0.209129  2.500000  2.000000
       Sat   0.147993  0.163817  2.307692  2.200000
       Sun   0.165710  0.237075  3.071429  2.500000
       Thur  0.155971  0.163073  2.480000  2.428571
Male   Fri   0.138005  0.144730  2.000000  2.125000
       Sat   0.162132  0.139067  2.656250  2.629630
       Sun   0.158291  0.173964  2.883721  2.600000
       Thur  0.165706  0.164417  2.500000  2.300000

In [69]:
tips.pivot_table(['tip_pct', 'size'], index=['sex', 'day'], columns='smoker', margins=True)

tip_pct                          size                    
smoker             No       Yes       All        No       Yes       All
sex    day                                                             
Female Fri   0.165296  0.209129  0.199388  2.500000  2.000000  2.111111
       Sat   0.147993  0.163817  0.156470  2.307692  2.200000  2.250000
       Sun   0.165710  0.237075  0.181569  3.071429  2.500000  2.944444
       Thur  0.155971  0.163073  0.157525  2.480000  2.428571  2.468750
Male   Fri   0.138005  0.144730  0.143385  2.000000  2.125000  2.100000
       Sat   0.162132  0.139067  0.151577  2.656250  2.629630  2.644068
       Sun   0.158291  0.173964  0.162344  2.883721  2.600000  2.810345
       Thur  0.165706  0.164417  0.165276  2.500000  2.300000  2.433333
All          0.159328  0.163196  0.160803  2.668874  2.408602  2.569672

In [72]:
tips.pivot_table(['tip_pct', 'size'], index=['sex', 'day'], columns='smoker', margins=True, aggfunc="count")

tip_pct                size             
smoker           No   Yes    All     No   Yes    All
sex    day                                          
Female Fri      2.0   7.0    9.0    2.0   7.0    9.0
       Sat     13.0  15.0   28.0   13.0  15.0   28.0
       Sun     14.0   4.0   18.0   14.0   4.0   18.0
       Thur    25.0   7.0   32.0   25.0   7.0   32.0
Male   Fri      2.0   8.0   10.0    2.0   8.0   10.0
       Sat     32.0  27.0   59.0   32.0  27.0   59.0
       Sun     43.0  15.0   58.0   43.0  15.0   58.0
       Thur    20.0  10.0   30.0   20.0  10.0   30.0
All           151.0  93.0  244.0  151.0  93.0  244.0

In [73]:
tips.pivot_table('size', index=['time', 'sex', 'smoker'], columns='day', aggfunc='sum', fill_value=0)

day                   Fri  Sat  Sun  Thur
time   sex    smoker                     
Dinner Female No        2   30   43     2
              Yes       8   33   10     0
       Male   No        4   85  124     0
              Yes      12   71   39     0
Lunch  Female No        3    0    0    60
              Yes       6    0    0    17
       Male   No        0    0    0    50
              Yes       5    0    0    23

In [ ]:
## <span style="color:blue">Example: Filling Missing Values with Group-specific Values</span>